# 옵티마이저 비교
옵티마이저 종류에는 RMSprop, Nadam , Adam, Adagrad가 있고 동일 조건 내에서 더 나은 옵티마이저를 찾는다.  
조건 :  
트레이닝-머니투데이190101200531 테스트-매일경제190101200531  
주가 등락 기준의 상한 및 하한-0.0077이상 상승,-0.005이하에서 하락, 그 사이를 유지라고 가정  
입력차원(사용 단어 수)-6000  
에포크-고정60에 ModelcheckPoint로 the highest test_accuracy인 지점의 모델 사용  
배치-전체 데이터 양의 1/5 ~1/6정도

In [1]:
# 사용할 데이터의 형태
data_type = '15'      # '1' : summarize X,개별 기사   '2' : summarize X,날짜별 기사
                    # '3' : summarize O,개별 기사   '4' : summarize O,날짜별 기사
normalized = 'done' # 'done' : 데이터 행별 normalize한 데이터     'not' : 데이터 행별 normalize 안한 데이터
erased_word = ''    # 제거된 단어 형태
num_word = '6000'   # 사용할 단어 수

# 돌려볼 모델의 구조
first_layer = 3000   # 모델의 첫 번째 레이어 수
second_layer = 1000  # 모델의 두 번째 레이어 수
ep = 60             # 모델의 에포크 수
ba = 1024            # 모델의 배치 사이즈

In [2]:
# 생성된 training, test의 x,y값을 pickle의 형태로 저장
import pickle
import os
path = os.getcwd()+'/pickles/'
if erased_word != '':
    erased_word = '_'+erased_word
with open(path+data_type+'train_x_'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'rb') as f:
    train_x = pickle.load(f)
with open(path+data_type+'train_y_'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'rb') as f:
    train_y = pickle.load(f)
with open(path+data_type+'train2_x_'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'rb') as f:
    train2_x = pickle.load(f)
with open(path+data_type+'train2_y_'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'rb') as f:
    train2_y = pickle.load(f)
with open(path+data_type+'test_x_'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'rb') as f:
    test_x = pickle.load(f)
with open(path+data_type+'test_y_'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'rb') as f:
    test_y = pickle.load(f)
with open(path+data_type+'test2_x_'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'rb') as f:
    test2_x = pickle.load(f)
with open(path+data_type+'test2_y_'+str(num_word)+'_'+normalized+erased_word+'.pickle', 'rb') as f:
    test2_y = pickle.load(f)

In [3]:
# numpy 사용하여 float로 바꾸어주는 과정
import numpy as np
x_train = np.asarray(train_x).astype('float32')
y_train = np.asarray(train_y).astype('float32')
x_test = np.asarray(test_x).astype('float32')
y_test = np.asarray(test_y).astype('float32')

In [15]:
# DNN 학습 모델
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import ModelCheckpoint


callbacks = [ModelCheckpoint(filepath='test',monitor='val_categorical_accuracy',save_best_only=True)]
model = models.Sequential()
n =len(x_train[0])
model.add(layers.Dense(first_layer,activation='relu'))#실제 쓸 단어갯수

model.add(layers.Dense(second_layer,activation='relu'))

model.add(layers.Dense(3,activation='softmax'))
model.compile(optimizer=optimizers.RMSprop(lr=0.001),loss = losses.categorical_crossentropy,metrics=[metrics.categorical_accuracy])

hist = model.fit(x_train,y_train,epochs=ep,validation_data=(x_test,y_test),callbacks=callbacks,batch_size=ba)
#hist = model.fit(x_train,y_train,epochs=ep,validation_data=(x_test,y_test),batch_size=ba)

# 모델의 예측값을 튜플로 반환하는 함수
def predict(x):
    data = np.expand_dims(np.asarray(x).astype('float32'),axis=0)
    result = model.predict(data)
    return tuple(result[0])


Epoch 1/60
5/5 [==============================] - 3s 594ms/step - loss: 1.0964 - categorical_accuracy: 0.3592 - val_loss: 1.0930 - val_categorical_accuracy: 0.3647
Epoch 2/60
5/5 [==============================] - 3s 564ms/step - loss: 1.0846 - categorical_accuracy: 0.3905 - val_loss: 1.0866 - val_categorical_accuracy: 0.4109
Epoch 3/60
5/5 [==============================] - 2s 408ms/step - loss: 1.0779 - categorical_accuracy: 0.4176 - val_loss: 1.0782 - val_categorical_accuracy: 0.3993
Epoch 4/60
5/5 [==============================] - 2s 410ms/step - loss: 1.0476 - categorical_accuracy: 0.4252 - val_loss: 1.1696 - val_categorical_accuracy: 0.3663
Epoch 5/60
5/5 [==============================] - 2s 455ms/step - loss: 1.0473 - categorical_accuracy: 0.4546 - val_loss: 1.0634 - val_categorical_accuracy: 0.4071
Epoch 6/60
5/5 [==============================] - 3s 611ms/step - loss: 0.9855 - categorical_accuracy: 0.5042 - val_loss: 1.0367 - val_categorical_accuracy: 0.4672
Epoch 7/60
5/5 [

5/5 [==============================] - 2s 450ms/step - loss: 0.2323 - categorical_accuracy: 0.9261 - val_loss: 1.4425 - val_categorical_accuracy: 0.5250
Epoch 42/60
5/5 [==============================] - 2s 451ms/step - loss: 0.3486 - categorical_accuracy: 0.8584 - val_loss: 1.2593 - val_categorical_accuracy: 0.4953
Epoch 43/60
5/5 [==============================] - 3s 625ms/step - loss: 0.2001 - categorical_accuracy: 0.9471 - val_loss: 1.4091 - val_categorical_accuracy: 0.5278
Epoch 44/60
5/5 [==============================] - 2s 426ms/step - loss: 0.1260 - categorical_accuracy: 0.9687 - val_loss: 1.5138 - val_categorical_accuracy: 0.5151
Epoch 45/60
5/5 [==============================] - 2s 450ms/step - loss: 0.2855 - categorical_accuracy: 0.8838 - val_loss: 2.0044 - val_categorical_accuracy: 0.4811
Epoch 46/60
5/5 [==============================] - 2s 447ms/step - loss: 0.2278 - categorical_accuracy: 0.9276 - val_loss: 1.5512 - val_categorical_accuracy: 0.5189
Epoch 47/60
5/5 [=====

In [16]:
model_best = models.load_model('test')

def b_predict(x):
    data = np.expand_dims(np.asarray(x).astype('float32'),axis=0)
    result = model_best.predict(data)
    return tuple(result[0])

In [6]:
# 모든 날짜의 예측값들을 합치기 위한 튜플의 합 함수

def sum_tuple(a,b):
    return (a[0]+b[0],a[1]+b[1],a[2]+b[2])

In [7]:
# Test set에 대한 예측 및 예측 정확도의 계산
#money190101을 트레이닝으로 삼은 모델로 final_maekyung190101예측결과) 0.0077 -0.005
#optimizer: Adam
import re
print('데이터 타입 : '+data_type)
print('단어 갯수 : '+num_word)
print('행당 노말라이즈 : '+ normalized)
print('모델 레이어 및 노드 수 : ','첫 번째 레이어 - relu,',first_layer,', 두 번째 레이어 - relu,',second_layer)
print('epochs : ',ep,', batch_size : ', ba)

count = 0
sum_dic = {}
for data in test2_x:
    result = b_predict(data[0])
    if data[1] in sum_dic:
        sum_dic[data[1]] = sum_tuple(sum_dic[data[1]],result)
    else:
        sum_dic[data[1]] = result
    
for date in sum_dic:
    real = test2_y[date]
    value = sum_dic[date].index(max(sum_dic[date]))
    value -= 1
    if real == value:
        count = count + 1
test_score = count / len(sum_dic)


#print('train 데이터 정확도 : ', train_score)
print('test 데이터 정확도 : ', test_score)

데이터 타입 : 15
단어 갯수 : 6000
행당 노말라이즈 : done
모델 레이어 및 노드 수 :  첫 번째 레이어 - relu, 3000 , 두 번째 레이어 - relu, 1000
epochs :  60 , batch_size :  1024
test 데이터 정확도 :  0.6829971181556196


In [11]:
# Test set에 대한 예측 및 예측 정확도의 계산
#money190101을 트레이닝으로 삼은 모델로 final_maekyung190101예측결과) 0.0077 -0.005
#optimizer: Nadam
import re
print('데이터 타입 : '+data_type)
print('단어 갯수 : '+num_word)
print('행당 노말라이즈 : '+ normalized)
print('모델 레이어 및 노드 수 : ','첫 번째 레이어 - relu,',first_layer,', 두 번째 레이어 - relu,',second_layer)
print('epochs : ',ep,', batch_size : ', ba)

count = 0
sum_dic = {}
for data in test2_x:
    result = b_predict(data[0])
    if data[1] in sum_dic:
        sum_dic[data[1]] = sum_tuple(sum_dic[data[1]],result)
    else:
        sum_dic[data[1]] = result
    
for date in sum_dic:
    real = test2_y[date]
    value = sum_dic[date].index(max(sum_dic[date]))
    value -= 1
    if real == value:
        count = count + 1
test_score = count / len(sum_dic)


#print('train 데이터 정확도 : ', train_score)
print('test 데이터 정확도 : ', test_score)

데이터 타입 : 15
단어 갯수 : 6000
행당 노말라이즈 : done
모델 레이어 및 노드 수 :  첫 번째 레이어 - relu, 3000 , 두 번째 레이어 - relu, 1000
epochs :  60 , batch_size :  1024
test 데이터 정확도 :  0.7435158501440923


In [14]:
# Test set에 대한 예측 및 예측 정확도의 계산
#money190101을 트레이닝으로 삼은 모델로 final_maekyung190101예측결과) 0.0077 -0.005
#optimizer: adagrad
import re
print('데이터 타입 : '+data_type)
print('단어 갯수 : '+num_word)
print('행당 노말라이즈 : '+ normalized)
print('모델 레이어 및 노드 수 : ','첫 번째 레이어 - relu,',first_layer,', 두 번째 레이어 - relu,',second_layer)
print('epochs : ',ep,', batch_size : ', ba)

count = 0
sum_dic = {}
for data in test2_x:
    result = b_predict(data[0])
    if data[1] in sum_dic:
        sum_dic[data[1]] = sum_tuple(sum_dic[data[1]],result)
    else:
        sum_dic[data[1]] = result
    
for date in sum_dic:
    real = test2_y[date]
    value = sum_dic[date].index(max(sum_dic[date]))
    value -= 1
    if real == value:
        count = count + 1
test_score = count / len(sum_dic)


#print('train 데이터 정확도 : ', train_score)
print('test 데이터 정확도 : ', test_score)

데이터 타입 : 15
단어 갯수 : 6000
행당 노말라이즈 : done
모델 레이어 및 노드 수 :  첫 번째 레이어 - relu, 3000 , 두 번째 레이어 - relu, 1000
epochs :  60 , batch_size :  1024
test 데이터 정확도 :  0.3400576368876081


In [17]:
# Test set에 대한 예측 및 예측 정확도의 계산
#money190101을 트레이닝으로 삼은 모델로 final_maekyung190101예측결과) 0.0077 -0.005
#optimizer :RMSprop
import re
print('데이터 타입 : '+data_type)
print('단어 갯수 : '+num_word)
print('행당 노말라이즈 : '+ normalized)
print('모델 레이어 및 노드 수 : ','첫 번째 레이어 - relu,',first_layer,', 두 번째 레이어 - relu,',second_layer)
print('epochs : ',ep,', batch_size : ', ba)

count = 0
sum_dic = {}
for data in test2_x:
    result = b_predict(data[0])
    if data[1] in sum_dic:
        sum_dic[data[1]] = sum_tuple(sum_dic[data[1]],result)
    else:
        sum_dic[data[1]] = result
    
for date in sum_dic:
    real = test2_y[date]
    value = sum_dic[date].index(max(sum_dic[date]))
    value -= 1
    if real == value:
        count = count + 1
test_score = count / len(sum_dic)


#print('train 데이터 정확도 : ', train_score)
print('test 데이터 정확도 : ', test_score)

데이터 타입 : 15
단어 갯수 : 6000
행당 노말라이즈 : done
모델 레이어 및 노드 수 :  첫 번째 레이어 - relu, 3000 , 두 번째 레이어 - relu, 1000
epochs :  60 , batch_size :  1024
test 데이터 정확도 :  0.7838616714697406
